In [ ]:
import Grammar
import Control.Monad.State
import Data.List.Extra (allSame, groupSortOn)

## Model

### Individuals

In [ ]:
domain :: [E]
domain = [john', mary', bill', thumper', bugs', h1', h2']
john' = "John"
mary' = "Mary"
bill' = "Bill"
thumper' = "Thumper"
bugs' = "Bugs"
h1' = "H1"
h2' = "H2"

### Properties

In [ ]:
predDomain :: [[E]]
predDomain = [boy', girl', rabbit', hat', brown']
boy' = [john', bill']
girl' = [mary']
rabbit' = [thumper', bugs']
hat' = [h1', h2']
brown' = [thumper']

### Relations

In [ ]:
relDomain :: [[(E,E)]]
relDomain = [likes', inside']
likes' = [(john', bill'), (john', mary'), (bill', mary')]
inside' = [(bugs', h1')]

## Language

### Names

In [ ]:
john, mary, bill, thumper, bugs, h1, h2 :: E
[john, mary, bill, thumper, bugs, h1, h2] = domain

### One-place predicates

In [ ]:
boy, girl, rabbit, hat, brown :: E -> T
[boy, girl, rabbit, hat, brown] = map (\xs x -> x `elem` xs) predDomain

### Two-place predicates

In [ ]:
likes, inside :: E -> E -> T
[likes, inside] = map (\rel x y -> (y,x) `elem` rel) relDomain

### Determiners

 Item | Type | Denotation
------|------|-----------
$\textbf{some}^u$ | $\mathbb{K}\,{\left(\mathbb{K}\,{\left(\mathbb{D}\,\rho\right)}\,{\left(\mathbb{D}\,\rho\right)}\,e\right)}\,{\left(\mathbb{D}\,t\right)}\,e$ | $\lambda cks.\, \cup\left\{k\,x\,s \mid x\in D_e,\ \langle \mathbb{T}, s'\rangle \in c\,x\,\widehat{sx}\right\}$
$\textbf{the}^u$ | $\mathbb{K}\,{\left(\mathbb{D}\,\varphi\right)}\,{\left(\mathbb{D}\,\varphi\right)}\,{\left(\mathbb{K}\,{\left(\mathbb{K}\,{\left(\mathbb{D}\,\rho\right)}\,{\left(\mathbb{D}\,\rho\right)}\,e\right)}\,{\left(\mathbb{D}\,t\right)}\,e\right)}$ | $\lambda ks.\, \textbf{1}_u\,{\left(k\,\textbf{some}_u\right)}\,s$

In [ ]:
some :: K (K (D r) (D r) E) (D T) E
some = Tower $ \p -> Tower $ \k -> some' p >>= k
  where some' :: (E -> D T) -> D E
        some' p = do x <- lift domain
                     p x >>= guard
                     state $ \s -> (x, s++[x])

the :: Int -> F r' -> K (D r') (D r') (K (K (D r) (D r) E) (D T) E)
the n t = Tower $ \c -> unique n . t $ c some
the' n = the n id

In [ ]:
-- Quantificational adjectives --------------------

unique :: Int -> F a
unique n = mapStateT exactlyOne
  where exactlyOne xs
          | null xs                      = errorWithoutStackTrace "existence failure"
          | allSame [s!!n | (_,s) <- xs] = xs
          | otherwise                    = errorWithoutStackTrace "uniqueness failure"

biggest :: Int -> F a
biggest n = mapStateT $ last . groupSortOn ((!!n) . snd)